# Week 3 Question 1

First, import the libs we'll need. then use wget to pull down the wikipedia page about Toroto Neighborhoods

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis 
import codecs
from bs4 import BeautifulSoup

!wget https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print("Done")

--2019-04-14 12:06:42--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.154.224, 2620:0:861:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80047 (78K) [text/html]
Saving to: ‘List_of_postal_codes_of_Canada:_M.3’

100%[======================================>] 80,047      --.-K/s   in 0.06s   

2019-04-14 12:06:43 (1.20 MB/s) - ‘List_of_postal_codes_of_Canada:_M.3’ saved [80047/80047]

Done


Now read the resulting file into a string
 - use Beautiful Soup to parse the neighborhood table into an object model 
 - grab just the table portion and turn that into a DataFrame, 
 - name the columns
 - and drop unassigned to clean the data

In [2]:
f = codecs.open('List_of_postal_codes_of_Canada:_M', encoding='utf-8')
html = f.read()
soup = BeautifulSoup(html, 'html.parser')

df = (pd.read_html(str(soup.table))[0])

#drop the first row, which is the headings of the table
df.drop(axis=0, index=0, inplace=True)

#then assign the proper column names
column_names = ['Postcode', 'Borough', 'Neighborhood'] 
df.columns = column_names

#get rid of unassigned Boroughs
df = df[df['Borough'] != "Not assigned"].reset_index(drop=True) 
df


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Next, there could be an unassigned neighborhood. That means give it the same name as its borough. Even though there is only one of these in the data set, other data sets could have more than one. So I'll set up a loop to fix it.

In [3]:
na = df[df['Neighborhood']=="Not assigned"]
for index in range(len(na)):
    r = na.iloc[index]
    r['Neighborhood']=r['Borough']
    df.iloc[na.index[0]]['Neighborhood'] = r['Borough']

df.head(8)    

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue


Next, there are multiple neighborhoods in many of the postcodes. And since we will be working the coordinates of postcodes, we need to make proper assignments of one to the4 other.
Do this by using a groupby with an aggregation lambda function that joins all of the neighborhood names into a single string. Use Merge to Now get all of the unique Post codes using Groupby, then walk through that df. 
This will result in multiple identical entries, which we can then use drop duplicates to get rid of
Finally, I will write the results to a file for use in the next part of the test


In [4]:
aggregations = {
    'Neighborhood':lambda x:', '.join(x)}
g = df.groupby('Postcode').agg(aggregations).reset_index()
x = df.drop('Neighborhood',axis=1)
m = pd.merge(x, g, on='Postcode', how='left')
dd = m.drop_duplicates()
dd.to_csv("Toronto_Q1_Answer.csv", index=False)
dd # dd means dropped duplicates    

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge, Malvern"
10,M3B,North York,Don Mills North
11,M4B,East York,"Woodbine Gardens, Parkview Hill"
13,M5B,Downtown Toronto,"Ryerson, Garden District"


In [5]:
dd.shape

(103, 3)